# More SQL - Solutions

This notebook builds on what we've learned previously (in Week 4 labs) by adding a few additional concepts, as well as some example SQL Interview Questions.  We'll be focusing on queries (`SELECT` statements).

## Example Interview Questions

The following are two real examples of SQL interview questions:

### Example Set 1: Customers and Orders

Done in real time.

Answer the questions about the following two tables:

```
customers                              orders

|---------------|---------|           |---------------|---------|
| id            | int     |<-+        | order_id      | int     |
| first_name    | varchar |  |-----+->| customer_id   | int     |
| last_name     | varchar |           | order_amount  | int     |
| zip_code      | varchar |           | order_date    | date    |
|---------------|---------|           |---------------|---------|
          
``` 
 
-- TIME LIMIT 15 MINUTES (GET THROUGH AS MUCH AS YOU CAN)

--1) Get all rows and columns from customers table

```sql
SELECT * 
FROM customers;
```

--2) Get all rows from customer table and only rows from orders table that match (Dupes are ok)

```sql
SELECT *
FROM customers c
INNER JOIN orders o ON c.id=o.customer_id;
```

--3) Get total order amount and total number of orders by customer name where their zip code is 98121 and order date is > 1/1/2015; if a customer has no orders results should show zero

```sql
 select c.first_name, c.last_name, sum(order_amount),count(*)
 from customers c
 left join orders o on c.id=o.customer_id
 where c.zip_code = 98121
 and order_date > '2015-01-01' 
 group by c.first_name, c.last_name;
```

--4) Using answer from #3, get only customers that had more than 2 orders

```sql
select c.first_name, c.last_name, sum(order_amount), count(order_id)
 from customers c
 left join orders o on c.id=o.customer_id
 where c.zip_code = 98121
 and order_date > '2015-01-01'
 and count(order_id)>2
 group by c.first_name, c.last_name;
```

--5) Using answer from #4 add in a numerical rank for each customer name ordering by the customer with the highest number of orders at the top

```sql
select firstname, lastname, invoices, rank( ) over ( order by invoices desc ) as rank
from (

select c.firstname, c.lastname, count(*) as invoices
from customers c
inner join invoices i on i.customerid = c.customerid
group by c.firstname, c.lastname

)
;
```

--6) Get the total number of orders broken into the following buckets based on total order amount: <\$5, \$5-\$20, >\$20

```sql
SELECT amt, count(*)
from

(SELECT c.customerid, count(*), 
CASE
     when sum(i.total) > 20 then '>$20'
     when sum(i.total) > 5 and sum(i.total) < 20 then '$5-$20'
     else '<$5'
end as amt
from customers c
inner join invoices i on i.customerid=c.customerid

group by c.customerid)
group by amt
;
```

--7) Get the second highest order amount per customer, if customer only has one order, order amount should show zero

```sql
select i.customerid, total as secondmax
from invoices i
where total < 
     (select max(total)
      from invoices ii
      where ii.customerid = i.customerid )
group by customerid;
```

### Example Set 2: Pizza Orders

Homework assignment.

Let’s say we have a database table of pizza orders.  Each row is one pizza that was ordered and let’s say that it includes these columns and this example data (the table is much larger, of course):

| order_id (INT) | order_date (DATE) | customer_id (INT) | order_item_id (INT) | canceled (CHAR(1)) | size (CHAR(3)) | toppings (VARCHAR(1024)) |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|10001|2016-01-01|9876|1|N|XL|
|10001|2016-01-01|9876|2|N|M|Pepperoni|
|10001|2016-01-01|9876|3|N|L|Pineapple, Ham|
|10003|2016-01-01|4658|1|Y|S|Black Olives, Spicy Peppers|
|10004|2016-01-02|3356|1|N|M|Bacon, Extra Cheese|
|10005|2016-01-03|7653|1|N|XL|Hamburger|
|10005|2016-01-03|7653|2|N|XL|Spicy Pepperoni, Feta Cheese

1.Write a SQL query that results in 1 row per month, with columns of:
- Month
- Total Orders
- Total Pizzas Ordered
- 1 more column for total count of each size that we sold (XL, L, M, S)

```sql
select pizza_totals.day_of_month, pizza_totals.total_orders, pizza_totals.total_pizzas,
 
		count(case when pizza_totals.size = 'S' then pizza_totals.total_pizzas else null end) as S,
        count(case when pizza_totals.size = 'M' then pizza_totals.total_pizzas else null end) as M,
        count(case when pizza_totals.size = 'L' then pizza_totals.total_pizzas else null end) as L,
        count(case when pizza_totals.size = 'XL' then pizza_totals.total_pizzas else null end) as XL

from
	(select extract(day from order_date) day_of_month, count(distinct order_id) total_orders, count(order_id) total_pizzas, size
		from pizza_orders
		where canceled != 'Y'
		group by extract(day from order_date), size
		order by extract(day from order_date)
     ) pizza_totals

group by pizza_totals.day_of_month, pizza_totals.total_orders, pizza_totals.total_pizzas
order by day_of_month
;
```

2.Write a SQL query that results in 1 row per customer and the following columns:
- Customer_id
- First order date
- Total pizzas ordered in the first order
- Whether any pizzas in the first order were canceled

```sql
select po.customer_id, po.order_date first_order_date, count(po.order_id) total_pizzas,
	(case when po.canceled = 'Y' then 'Yes' else 'No' end) were_pizzas_canceled
from pizza_orders po
	
where

	po.order_date = (select min(po2.order_date)
                    	from pizza_orders po2
                    	where po.customer_id = po2.customer_id)


group by po.customer_id, po.order_id, po.canceled, po.order_date;
```

3.Write a SQL query to see how well our new Spicy Pepperoni topping is selling.

```sql
select extract(month from order_date) as month, pizzas_spicy_pepperoni/total_pizzas_per_day as percentage_pizzas_spicy_pepperoni
from
(select order_date, extract(day from order_date) as day,  
		count(distinct order_id) as total_orders_per_day,
		count(order_id) as total_pizzas_per_day,
		sum(case when toppings like '%Spicy Pepperoni%' then 1 else 0 end) as pizzas_spicy_pepperoni
		from pizza_orders
		group by order_date,day) as pizza_orders
group by month, percentage_pizzas_spicy_pepperoni
;
```

## More references

A slightly more advanced tutorial: https://www.tutorialspoint.com/sql/index.htm

A more interactive tutorial, with different sample databases: https://community.modeanalytics.com/sql/

http://sqlfiddle.com/#!9/9eecb

W3Schools https://www.w3schools.com/sql/default.asp

Join types: http://www.sql-join.com/sql-join-types/